# Webscraping Scientific Articles

In [ ]:
!pip install requests beautifulsoup4 selenium webdriver-manager

In [ ]:
import os
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait


### Section 1: Using an API (PubMed)

The first method we'll explore for web scraping is using an API (Application Programming Interface). APIs are interfaces provided by many web services that allow developers to interact with their service in a structured manner. They often provide data in a machine-readable format like JSON or XML, which is easier to work with than raw HTML.

In this section, we'll be using the PubMed API to search for scientific papers. PubMed is a free search engine that provides access to the MEDLINE database of references and abstracts on life sciences and biomedical topics.

The process of using an API typically involves sending a GET request to a specific URL (often called an endpoint), with parameters that specify the data you're interested in. The server then responds with the data in the format specified (JSON or XML).

The function `pubmed_search` below demonstrates how to use the PubMed API to search for papers. The search terms and the maximum number of papers to return are specified as parameters to the function.


In [ ]:
ESEARCH_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
EFETCH_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

def pubmed_search(search_terms, retmax):
    # Combine search terms with AND operator
    query = " AND ".join(search_terms)

    # Set up the URL and parameters for esearch
    esearch_params = {
        'db': 'pubmed',
        'term': query,
        'retmode': 'json',
        'retmax': retmax
    }

    # Call esearch
    print("Now performing esearch...")
    esearch_response = requests.get(ESEARCH_URL, params=esearch_params)
    esearch_data = esearch_response.json()

    # Check for 'esearchresult' in the response
    if 'esearchresult' in esearch_data:
        # Extract UIDs (PMIDs)
        uid_list = esearch_data['esearchresult']['idlist']

        # Check if there are any UIDs
        if not uid_list:
            print("No search results found.")
            return

        # For each UID, fetch the corresponding XML data
        for uid in uid_list:
            # Set up the URL and parameters for efetch
            efetch_params = {
                'db': 'pubmed',
                'id': uid,
                'retmode': 'xml',
                'rettype': 'full'
            }

            # Call efetch
            print(f"Now performing efetch for UID {uid}...")
            efetch_response = requests.get(EFETCH_URL, params=efetch_params)
            xml_data = efetch_response.text

            os.makedirs(os.path.join(os.getcwd(), "PubMed"), exist_ok=True)
            
            # Save the XML data to a file
            with open(os.path.join("PubMed", f"{uid}.xml"), 'w') as f:
                f.write(xml_data)

            # Sleep for 1/2 of a second to avoid hitting the rate limit
            time.sleep(1 / 2)


Now that we have this function defined, we can go ahead and call it with a list of search terms, followed by the number to return. Then, if you open up the folder this notebook is in, you will see a new folder named 'PubMed' and it will contain the documents we scraped.

In [ ]:
pubmed_search(["AI", "Machine Learning"], 5)


### Section 2: Parsing HTML (arXiv)

HTML parsing involves downloading the HTML content of a webpage and extracting useful information from it. This method is used when the website does not provide an API, or the API does not provide the data we need. 

In this section, we will demonstrate how to parse HTML using the BeautifulSoup library. BeautifulSoup provides a few simple methods and Pythonic idioms for navigating, searching, and modifying a parse tree. 

We will use the arXiv preprint server as an example, arXiv is an open-access repository of scientific papers in the fields of physics, mathematics, computer science, and more. 


In [ ]:
def sanitize_filename(filename):
    keepcharacters = (' ', '.', '_', '-')
    return "".join(c for c in filename if c.isalnum() or c in keepcharacters).rstrip()

In [ ]:
def arxiv_search(search_terms, retmax):
    # Combine search terms with AND operator
    query = "+AND+".join(search_terms)

    # Set up the URL for arXiv search
    arxiv_url = f"https://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results={retmax}"

    # Get the HTML content of the webpage
    response = requests.get(arxiv_url)
    xml_data = response.text

    # Parse the XML data using BeautifulSoup
    soup = BeautifulSoup(xml_data, "xml")

    # Create a directory to store the downloaded PDFs if it doesn't exist
    pdf_dir = os.path.join(os.getcwd(), 'arXiv')
    os.makedirs(pdf_dir, exist_ok=True)

    # Find all papers
    entries = soup.find_all("entry")
    for entry in entries:
        # Extract the title, author(s), and link of each paper
        title = entry.title.text
        authors = [author.find("name").text for author in entry.find_all("author")]
        link = entry.id.text

        print(f"Title: {title}")
        print(f"Authors: {authors}")
        print(f"Link: {link}")
        print("\n")

        # Convert the link to the PDF download link
        pdf_link = link.replace("abs", "pdf")
        pdf_link = urljoin(pdf_link, pdf_link.split("/")[-1] + ".pdf")

        # Download the PDF
        pdf_response = requests.get(pdf_link)
        
        # Sanitize the title to be used as a filename
        sanitized_title = sanitize_filename(title)
        pdf_filename = os.path.join(pdf_dir, f"{sanitized_title}.pdf")
        with open(pdf_filename, 'wb') as f:
            f.write(pdf_response.content)

        print(f"PDF downloaded to: {pdf_filename}")


Again, now that our function is defined, we can go ahead and call it in the exact same way as before.

In [ ]:
arxiv_search(["AI", "Machine Learning"], 5)


### Section 3: Headless Browsing (ScienceOpen)

Headless browsing is a technique in which a web browser runs without a graphical user interface. This is useful for automating tasks on websites that require user interaction, or on websites that load content dynamically with JavaScript. 

In this section, we will demonstrate how to perform headless browsing using Selenium. Selenium is a powerful tool for controlling a web browser through the program. It is functional for all browsers, works on all major OS and its scripts can be written in various programming languages such as Python, Java, C#, etc.

We will use ScienceOpen as an example. ScienceOpen is a professional networking platform for scholars to share and evaluate scientific information.

Please note that Selenium requires a driver to interface with the chosen browser. The driver for each browser needs to be installed and path should be set in the environment variables. 


In [ ]:
import os
import platform
import zipfile
import urllib.request
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

def get_chromedriver_path():
    chrome_version = "114.0.5735.90"
    os_type = platform.system()

    if os_type == 'Windows':
        zip_filename = "chromedriver_win32.zip"
    elif os_type == 'Darwin':
        if platform.processor() == 'arm':
            zip_filename = "chromedriver_mac_arm64.zip"
        else:
            zip_filename = "chromedriver_mac64.zip"
    elif os_type == 'Linux':
        zip_filename = "chromedriver_linux64.zip"
    else:
        raise ValueError(f"Unsupported OS: {os_type}")

    zip_url = f"https://chromedriver.storage.googleapis.com/{chrome_version}/{zip_filename}"
    zip_path = os.path.join(os.getcwd(), zip_filename)
    chromedriver_path = os.path.join(os.getcwd(), 'chromedriver')

    if not os.path.exists(chromedriver_path):
        # Download the zip file
        urllib.request.urlretrieve(zip_url, zip_path)

        # Extract the ChromeDriver executable
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(os.getcwd())

        # Remove the zip file
        os.remove(zip_path)

        if os_type != 'Windows':
            # Make the ChromeDriver executable
            os.chmod(chromedriver_path, 0o755)

    return chromedriver_path


In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import logging
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from selenium.common.exceptions import StaleElementReferenceException
from tqdm import tqdm
from bs4 import BeautifulSoup
from urllib.parse import quote
import requests


def scienceopen_search(search_terms, retmax):
    # Make sure the chromdriver is installed
    webdriver_service = Service(get_chromedriver_path())
    
    # Turn off the ridiculous amount of logging selenium does
    LOGGER = logging.getLogger()
    LOGGER.setLevel(logging.WARNING)

    # Setup Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Ensure GUI is off
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-crash-reporter")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-in-process-stack-traces")
    chrome_options.add_argument("--disable-logging")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--log-level=3")
    chrome_options.add_argument("--output=/dev/null")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

    # Choose Chrome Browser
    driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)
    wait = WebDriverWait(driver, 10)

    url = f"https://www.scienceopen.com/search#('v'~4_'id'~''_'queryType'~1_'context'~null_'kind'~77_'order'~0_'orderLowestFirst'~false_'query'~'{' '.join(search_terms)}'_'filters'~!('kind'~84_'openAccess'~true)*_'hideOthers'~false)"
    driver.get(url)

    # Create a directory to store the scraped links if it doesn't exist
    scraped_links_dir = os.path.join(os.getcwd(), 'SO_searches')
    os.makedirs(scraped_links_dir, exist_ok=True)

    # Create a file to store the scraped links
    scraped_links_file_path = os.path.join(scraped_links_dir, "scraped_links.txt")

    # Load the already scraped links
    if os.path.exists(scraped_links_file_path):
        with open(scraped_links_file_path, 'r') as file:
            scraped_links = file.read().splitlines()
    else:
        scraped_links = []

    article_links = []
    while len(article_links) < retmax:
        # Extract all the article links
        new_links = driver.find_elements(By.CSS_SELECTOR, 'div.so-article-list-item > div > h3 > a')
        new_links = [link.get_attribute('href') for link in new_links if
                     link.get_attribute('href') not in scraped_links]

        # Filter out the already scraped links
        article_links.extend(new_links)

        # Click the "Load More" button
        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.so--tall'))
            )
            driver.execute_script("arguments[0].click();", load_more_button)
            time.sleep(2)
        except TimeoutException:
            break
        except StaleElementReferenceException:
            print("No more results to load.")
            break
        except Exception as other:
            print(f"An unknown exception occurred, please let the dev know: {other}")

    start_time = time.time()
    pbar = tqdm(total=retmax, dynamic_ncols=True)
    count = 0
    for link in article_links:
        if count >= retmax:
            break

        # Open a new tab and switch to it
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[-1])

        try:
            # Visit the article page
            driver.get(link)

            # Extract the PDF download link
            pdf_link_element = wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#id2e > li:nth-child(1) > a:nth-child(1)')))
            pdf_link = pdf_link_element.get_attribute('href')

            # Extract the DOI
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            doi_element = soup.find('meta', attrs={'name': 'citation_doi'})
            if doi_element is not None:
                doi = doi_element.get('content')
                encoded_doi = quote(doi, safe='')

                # Download the PDF
                pdf_response = requests.get(pdf_link)
                filename = f"{encoded_doi}.pdf"
                with open(os.path.join(os.getcwd(), 'SO_searches', filename), 'wb') as f:
                    f.write(pdf_response.content)

                count += 1
                elapsed_time = time.time() - start_time
                avg_time_per_pdf = elapsed_time / count
                est_time_remaining = avg_time_per_pdf * (retmax - count)
                pbar.set_description(
                    f"DOI: {doi}, Count: {count}/{retmax}, Avg time per PDF: {avg_time_per_pdf:.2f}s, Est. time remaining: {est_time_remaining:.2f}s")
                pbar.update(1)

                # Append the scraped link to the file
                with open(scraped_links_file_path, 'a') as file:
                    file.write(f"{link}\n")
        except Exception as e:
            print(f"An error occurred while processing the article: {e}")
        finally:
            # Close the tab and switch back to the original tab
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

    driver.switch_to.window(driver.window_handles[0])

    driver.quit()
    pbar.close()


In [ ]:
scienceopen_search(["AI", "Machine Learning"], 5)